In [2]:
from future.builtins import next
import re
from numpy import nan
import dedupe
from unidecode import unidecode
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
import json
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType,FloatType
import warnings
warnings.simplefilter(action='ignore')

In [3]:
spark = SparkSession \
    .builder \
    .appName("dedupe") \
    .master("local[*]") \
    .getOrCreate()

22/08/03 14:20:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [10]:
input_file = '/home/abhishekks446/data/netflix_titles.csv'


primart_key="unique_id"


df1= spark.read.option("header","true").csv(input_file).limit(2000)

In [11]:
df1.count()

2000

In [12]:
def preProcess(column):
    """
    Do a little bit of data cleaning with the help of Unidecode and Regex.
    Things like casing, extra spaces, quotes and new lines can be ignored.
    """
    import unidecode
    # column = column.decode("utf8")
    column = unidecode.unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
    if not column :
        column = None
    return column
def readData():
    """
    Read in our data from a CSV file and create a dictionary of records,
    where the key is a unique record ID and each value is dict
    """
    
    data_d = {}
    dfn=df1.na.fill("null")
    results = dfn.toJSON().map(lambda j: json.loads(j)).collect()
    for row in results:
        clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
        row_id = row[primart_key]
        data_d[row_id] = dict(clean_row)

    return data_d
data_d = readData()

In [13]:
print(data_d)

{'s1': {'unique_id': 's1', 'type': 'movie', 'title': 'dick johnson is dead', 'director': 'kirsten johnson', 'cast': 'null', 'country': 'united states', 'date_added': 'september 25, 2021', 'release_year': '2020', 'rating': 'pg-13', 'duration': '90 min', 'listed_in': 'documentaries', 'description': 'as her father nears the end of his life, filmmaker kirsten johnson stages his death in inventive and comical ways to help them both face the inevitable.'}, 's2': {'unique_id': 's2', 'type': 'tv show', 'title': 'blood & water', 'director': 'null', 'cast': 'ama qamata, khosi ngema, gail mabalane, thabang molaba, dillon windvogel, natasha thahane, arno greeff, xolile tshabalala, getmore sithole, cindy mahlangu, ryle de morny, greteli fincham, sello maake ka-ncube, odwa gwanya, mekaila mathys, sandi schultz, duane williams, shamilla miller, patrick mofokeng', 'country': 'south africa', 'date_added': 'september 24, 2021', 'release_year': '2021', 'rating': 'tv-ma', 'duration': '2 seasons', 'listed_

In [14]:
fields = [
{'field' : 'type', 'type': 'String'},
{'field' : 'title', 'type': 'String'}
]
deduper = dedupe.Dedupe(fields)

In [15]:
deduper.prepare_training(data_d)

In [16]:
pair = deduper.uncertain_pairs()
print(pair)

[({'unique_id': 's331', 'type': 'movie', 'title': 'cloudy with a chance of meatballs', 'director': 'phil lord, christopher miller', 'cast': "bill hader, anna faris, james caan, andy samberg, bruce campbell, mr. t, bobb'e j. thompson, benjamin bratt, neil patrick harris, al roker", 'country': 'united states, canada', 'date_added': 'august 1, 2021', 'release_year': '2009', 'rating': 'pg', 'duration': '90 min', 'listed_in': 'children & family movies, comedies, sci-fi & fantasy', 'description': 'when inventor flint lockwood makes clouds rain food, the citizens of chewandswallow can feed themselves. but a bowl of disaster is about to overflow.'}, {'unique_id': 's1695', 'type': 'movie', 'title': 'cloudy with a chance of meatballs 2', 'director': 'cody cameron, kris pearn', 'cast': 'bill hader, anna faris, james caan, will forte, andy samberg, benjamin bratt, neil patrick harris, terry crews, kristen schaal', 'country': 'united states', 'date_added': 'november 15, 2020', 'release_year': '2013